# Faiss

Facebook AI Similarity Search (Faiss)는 밀집 벡터의 효율적인 유사도 검색과 클러스터링을 위한 라이브러리입니다.

Faiss는 RAM에 맞지 않을 수도 있는 벡터 집합을 포함하여 모든 크기의 벡터 집합을 검색하는 알고리즘을 포함하고 있습니다.

또한 평가와 매개변수 튜닝을 위한 지원 코드도 포함되어 있습니다.

[FAISS 문서](https://faiss.ai/)를 참조하세요.


## 설치

langchain-community 패키지에 통합 기능이 포함되어 있습니다. 또한 faiss 패키지 자체도 설치해야 합니다.

임베딩에는 `OpenAIEmbeddings` 를 사용할 예정이므로 해당 요구사항도 설치해야 합니다.

GPU 지원 버전을 사용하려면 faiss-gpu를 설치할 수도 있습니다.


In [ ]:
%pip install -U langchain-community faiss-cpu langchain-openai tiktoken

In [ ]:
import os

# 사용자로부터 OpenAI API 키를 입력받아 환경 변수로 설정합니다.
os.environ["OPENAI_API_KEY"] = "OPENAI API KEY 입력"

혹은 `dot_env` 라이브러리를 사용하여 `.env` 파일을 만들어 환경 변수로 설정할 수 있습니다.


In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

추적을 위한 LangSmith 설정(필수는 아닙니다.)


In [1]:
# LANGCHAIN_TRACING_V2 환경 변수를 "true"로 설정합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# LANGCHAIN_API_KEY 환경 변수를 getpass.getpass() 함수를 통해 입력받은 값으로 설정합니다.
# os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN API KEY 입력"

## DB에 저장

여기에서는 문서를 벡터 저장소에 수집합니다.


- `TextLoader`를 사용하여 텍스트 데이터를 로드합니다.
- `CharacterTextSplitter`를 사용하여 로드된 문서를 1000자 단위로 분할하고, 분할된 문서 간에 중복되는 내용이 없도록 설정합니다.
- `OpenAIEmbeddings`를 사용하여 문서 임베딩을 생성합니다.
- `FAISS` 벡터 저장소를 초기화하고, 분할된 문서와 임베딩을 사용하여 벡터 인덱스를 구축합니다.

**참고**

- `AVX2`: 고도의 병렬 처리가 가능한 연산을 사용하는 벡터화 가능 알고리즘의 경우 `AVX2` 를 사용하면 CPU 성능이 향상되어 지연 시간이 줄어들며 처리량이 향상됩니다.
- 필요한 경우 `os.environ['FAISS_NO_AVX2'] = '1'` 코드 라인의 주석을 해제하여 FAISS에서 AVX2 최적화를 사용하지 않도록 설정할 수 있습니다.


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# FAISS에서 AVX2 최적화를 사용하지 않으려면 다음 줄의 주석을 해제하세요.
# import os
#
# os.environ['FAISS_NO_AVX2'] = '1'

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)

# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(docs, embeddings)

## 쿼리

DB 에 데이터를 저장했으므로, 이제 우리는 vectorstore를 쿼리할 수 있습니다.

이를 수행하는 몇 가지 방법이 있는데, 가장 일반적인 방법은 `similarity_search`를 사용하는 것입니다.

`similarity_search` 메서드는 주어진 쿼리와 유사한 벡터를 vectorstore에서 검색하여 가장 유사한 결과를 반환합니다.


쿼리를 위하여 일부 파일의 내용을 확인합니다.


In [5]:
# data/appendix-keywords.txt 파일 내용을 읽어서 file 변수에 저장합니다.
with open("./data/appendix-keywords.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

# file 변수에 저장된 내용을 출력합니다.
print(file[:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어


`db.similarity_search(query)`를 호출하여 질문과 유사한 문서를 검색합니다.


In [6]:
# 쿼리(query) 변수에 저장된 질문과 유사한 문서를 데이터베이스에서 검색합니다.
query = "임베딩(Embedding)이란 무엇인가요?"
docs = db.similarity_search(query)  # 질문과 유사한 문서를 데이터베이스에서 검색

쿼리에 대한 유사도가 가장 높은 문서를 출력합니다.


In [7]:
# docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(docs[0].page_content)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


## Retriever로 활용

vectorstore를 [Retriever](/docs/modules/data_connection/retrievers) 클래스로 변환할 수도 있습니다.


`db.as_retriever()` 메서드를 호출하여 데이터베이스를 검색기(retriever)로 사용할 수 있는 객체를 생성합니다.

- 이 메서드는 데이터베이스 객체 `db`를 검색기로 변환합니다.
- 반환된 `retriever` 객체는 질의에 대한 관련 문서를 데이터베이스에서 검색하는 데 사용됩니다.


In [8]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당합니다.
retriever = db.as_retriever()

`retriever.invoke(query)`는 검색 질의 `query`를 사용하여 `retriever` 객체에서 관련 문서를 검색합니다.

- `retriever`는 벡터 저장소 또는 검색 엔진과 같은 문서 검색 시스템을 나타내는 객체입니다.
- `invoke` 메서드는 `retriever` 객체에 정의된 검색 기능을 호출하여 질의에 관련된 문서를 검색합니다.


In [10]:
# 검색 질의를 사용하여 관련 문서를 검색합니다.
query = "임베딩(Embedding)이란 무엇인가요?"
docs = retriever.invoke(query)

반환된 결과를 확인합니다.


In [12]:
# docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(docs[0].page_content)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


## 점수에 기반한 유사도 검색

FAISS에는 몇 가지 특정 메서드가 있습니다.

그 중 하나는 `similarity_search_with_score`로, 문서뿐만 아니라 쿼리와 문서 간의 거리 점수도 반환할 수 있습니다.

반환되는 거리 점수는 L2 거리입니다. 따라서 **점수가 낮을수록 더 좋은 결과** 입니다.


`db.similarity_search_with_score(query)` 메서드를 사용하여 질의와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.

- 반환된 결과는 (문서, 점수) 튜플의 리스트 형태로 구성됩니다.


In [18]:
# 쿼리와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.
docs_and_scores = db.similarity_search_with_score(query)
content, score = docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다
print("[Content]")
print(content.page_content)  # 선택된 문서의 page_content 속성을 출력합니다
print("\n[Score]")
print(score)  # 선택된 문서의 점수를 출력합니다

[Content]
정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

[Score]
0.29661512


`similarity_search_by_vector` 함수를 사용하면 주어진 임베딩 벡터와 유사한 문서를 검색할 수 있습니다.

이 함수는 문자열 대신 **임베딩 벡터** 를 매개변수로 받아들입니다.


In [24]:
# 질의를 임베딩 벡터로 변환합니다.
query = "임베딩(Embedding)이란 무엇인가요?"
embedding_vector = embeddings.embed_query(query)
# 임베딩 벡터를 사용하여 유사도 검색을 수행하고, 문서와 점수를 반환합니다.
docs_and_scores = db.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다

Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'})

## 저장 및 로드

FAISS 인덱스를 저장하고 불러올 수도 있습니다.

이는 인덱스를 사용할 때마다 매번 다시 생성할 필요가 없어 유용합니다.


- `db.save_local("저장할 인덱스 이름")`를 사용하여 FAISS 인덱스를 로컬 디렉토리에 저장합니다.


In [26]:
# 로컬에 "MY_FIRST_DB_INDEX"라는 이름으로 데이터베이스를 저장합니다.
DB_INDEX = "MY_FIRST_DB_INDEX"
db.save_local(DB_INDEX)

- `FAISS.load_local("불러올 인덱스 이름", embeddings)`를 사용하여 저장된 FAISS 인덱스를 로드합니다.


In [31]:
# 로컬에 저장된 데이터베이스를 불러와 new_db 변수에 할당합니다.
new_db = FAISS.load_local(DB_INDEX, embeddings,
                          allow_dangerous_deserialization=True)

query = "임베딩(Embedding)이란 무엇인가요?"

# new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
docs = new_db.similarity_search(query)

# 문서 리스트의 첫 번째 문서를 가져옵니다.
docs[0]

Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'})

# 바이트 형태로 직렬화/역직렬화

FAISS 인덱스를 직렬화(Serializing)하고 역직렬화(De-Serializing)하는 데에는 `pickle` 함수를 사용하여 저장할 수 있습니다. 하지만, 만약 90MB 크기의 임베딩 모델(예: `sentence-transformers/all-MiniLM-L6-v2` 또는 다른 모델)을 사용한다면, 결과로 생성되는 pickle 파일의 크기는 90MB 이상이 될 것입니다. 이는 모델의 크기 또한 전체 크기에 포함되기 때문입니다.

따라서, 아래의 함수들을 사용하는 것이 좋습니다. 이 함수들은 **FAISS 인덱스만 직렬화** 하므로 크기가 훨씬 작아집니다.


In [79]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)

# HuggingFaceEmbeddings 사용하여 임베딩을 생성합니다.
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(docs, hf_embeddings)

`serialize_to_bytes` 함수로 DB 인덱스를 직렬화합니다.


In [80]:
# FAISS 인덱스를 직렬화합니다.
serialized_db_index = db.serialize_to_bytes()

직렬화된 `serialized_db_index` 의 사이즈를 확인합니다.


In [81]:
import sys

# 사이즈 측정을 위한 함수를 정의합니다.


def get_size(path):
    size = sys.getsizeof(path)
    if size < 1024:
        return f"{size} bytes"
    elif size < pow(1024, 2):
        return f"{round(size/1024, 2)} KB"
    elif size < pow(1024, 3):
        return f"{round(size/(pow(1024,2)), 2)} MB"
    elif size < pow(1024, 4):
        return f"{round(size/(pow(1024,3)), 2)} GB"


# 직렬화된 FAISS 인덱스의 사이즈를 출력합니다.
get_size(serialized_db_index)

'60.81 KB'

이번에는 `deserialize_from_bytes` 함수로 역직렬화를 수행합니다.


In [82]:
# 직렬화된 인덱스를 로드합니다.
deserialized_db = FAISS.deserialize_from_bytes(
    embeddings=hf_embeddings,  # 직렬화 할 때의 임베딩과 동일하게 지정
    serialized=serialized_db_index,  # 직렬화된 인덱스
)

In [83]:
query = "임베딩(Embedding)이란 무엇인가요?"

# new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
docs = deserialized_db.similarity_search(query)

# 문서 리스트의 첫 번째 문서를 가져옵니다.
docs[0]

Document(page_content='정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer', metadata={'source': './data/appendix-keywords.txt'})

## 병합(merge)

FAISS 벡터 저장소를 병합하는 것도 가능합니다.

두 개의 FAISS 벡터 저장소를 하나로 합칠 수 있습니다.


In [93]:
# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# db1 생성
db1 = FAISS.from_texts(["LangChain!!"], embeddings)
# db2 생성
db2 = FAISS.from_texts(["좋아요^^"], embeddings)

`db1` 의 내용 확인


In [94]:
# db1 벡터 데이터베이스의 내부 문서 저장소 딕셔너리에 접근합니다.
db1.docstore._dict

{'869410ed-8353-4deb-9517-4fca77e291cc': Document(page_content='LangChain!!')}

`db2` 의 내용 확인


In [95]:
# db2 문서 저장소의 내부 딕셔너리에 접근합니다.
db2.docstore._dict

{'5695551b-a697-4ed5-bbf4-ae9fff86f51f': Document(page_content='좋아요^^')}

`db1` 객체의 `merge_from` 메서드를 사용하여 `db2` 객체의 내용을 `db1`에 병합합니다.

- `merge_from` 메서드는 `db2`의 데이터를 `db1`에 병합하는 기능을 수행합니다.
- 병합 작업 후에는 `db1`에 `db2`의 데이터가 포함됩니다.


In [96]:
db1.merge_from(db2)  # db1에 db2를 병합합니다.

병합된 결과를 확인합니다.


In [97]:
# db1 문서 저장소의 내부 딕셔너리에 접근합니다.
db1.docstore._dict

{'869410ed-8353-4deb-9517-4fca77e291cc': Document(page_content='LangChain!!'),
 '5695551b-a697-4ed5-bbf4-ae9fff86f51f': Document(page_content='좋아요^^')}

## 필터링

FAISS vectorstore는 필터링 기능도 지원할 수 있습니다.

FAISS는 기본적으로 필터링을 지원하지 않기 때문에, 이를 수동으로 처리해야 합니다.

**방식**

1. 이는 먼저 `k`보다 더 많은 결과를 가져온 다음 필터링하는 방식으로 이루어집니다.

2. 이 필터는 메타데이터 dict를 입력으로 받아 bool을 반환하는 호출 가능한 함수이거나, 누락된 각 키는 무시되고 존재하는 각 키는 값 목록에 있어야 하는 메타데이터 dict입니다.

또한 검색 메서드를 호출할 때 `fetch_k` 매개변수를 설정하여 필터링 전에 가져올 문서 수를 지정할 수 있습니다.


In [98]:
from langchain_core.documents import Document

list_of_documents = [
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="foo", metadata=dict(page=1)),
    # 페이지 내용이 "bar"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="bar", metadata=dict(page=1)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="foo", metadata=dict(page=2)),
    # 페이지 내용이 "barbar"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="barbar", metadata=dict(page=2)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="foo", metadata=dict(page=3)),
    # 페이지 내용이 "bar burr"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="bar burr", metadata=dict(page=3)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="foo", metadata=dict(page=4)),
    # 페이지 내용이 "bar bruh"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
# 문서 리스트와 임베딩을 사용하여 FAISS 데이터베이스 생성
db = FAISS.from_documents(list_of_documents, embeddings)

# "foo"와 유사한 문서를 검색하고 점수와 함께 결과 반환
results_with_scores = db.similarity_search_with_score("foo")

for doc, score in results_with_scores:  # 검색 결과를 반복하면서
    # 각 문서의 내용, 메타데이터, 점수를 출력
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 0.0
Content: foo, Metadata: {'page': 2}, Score: 0.0
Content: foo, Metadata: {'page': 3}, Score: 0.0
Content: foo, Metadata: {'page': 4}, Score: 0.0


이제 동일한 쿼리 호출을 수행하지만, `page = 1`인 경우만 필터링합니다.


- `filter` 매개변수를 사용하여 검색 결과를 필터링할 수 있습니다. 딕셔너리 형태로 메타데이터 필드와 값을 지정하거나, 호출 가능한 객체(callable)를 사용할 수 있습니다.
- 검색 결과는 `(문서, 점수)` 튜플의 리스트 형태로 반환됩니다.


In [102]:
# 방법1) 유사도 검색을 수행하고 필터를 적용하여 결과와 점수를 반환합니다.
results_with_scores = db.similarity_search_with_score(
    "foo", filter=dict(page=1))
# 방법2) 혹은 callable 을 사용하여 필터링 하는 경우
# results_with_scores = db.similarity_search_with_score("foo", filter=lambda d: d["page"] == 1)

for doc, score in results_with_scores:  # 결과와 점수를 반복합니다.
    # 각 문서의 내용, 메타데이터, 점수를 출력합니다.
    print(
        f"[Content] {doc.page_content}, [metadata] {doc.metadata}, [Score] {score}")

[Content] foo, [metadata] {'page': 1}, [Score] 0.0
[Content] bar, [metadata] {'page': 1}, [Score] 0.31470414996147156


## Max Marginal Relevance (MMR)

`MMR(Maximal Marginal Relevance)` 방식은 쿼리에 대한 관련 항목을 검색할 때 중복을 피하는 방법 중 하나입니다. 단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 검색된 항목들 사이에 **관련성과 다양성 사이의 균형을 보장**합니다. 이는 **자주 발생할 수 있는, 매우 유사한 항목들만이 검색되는 상황을 방지** 하는 데에 유용합니다.

예를 들어, 특정 주제에 대해 정보를 찾고 있다고 가정해봅시다. 가장 관련성 높은 문서만을 반환하는 시스템은 비슷비슷한 정보를 담은 문서들을 제공할 수 있습니다. 하지만, MMR 방식을 사용하면, 검색된 문서들이 해당 주제에 대해 서로 다른 관점이나 새로운 정보를 제공하도록 합니다. 이로써 사용자는 주제에 대해 보다 폭넓은 이해를 할 수 있게 됩니다.

MMR은 두 가지 주요 요소, 즉 쿼리에 대한 문서의 관련성과 **이미 선택된 문서들과의 차별성을 동시에 고려** 합니다.

1. 첫째로, 쿼리와의 관련성이 높은 문서를 찾는 것이 중요합니다.
2. 둘째로, 이미 선택된 문서들과는 다른 새로운 정보나 관점을 제공하는 문서를 찾는 것입니다.

이 두 가지 요소의 균형을 맞추는 것이 MMR의 핵심입니다.


`max_marginal_relevance_search` 함수에도 동일한 방식을 적용할 수 있습니다.


- `db.max_marginal_relevance_search()` 메서드를 사용하여 "foo"라는 검색어로 문서를 검색합니다.
- 이때 `filter` 매개변수를 사용하여 `page` 메타데이터 값이 1인 문서만 검색합니다.


In [105]:
# "foo"를 검색어로 사용하여 최대 한계 관련성 검색을 수행하고, 메타데이터의 'page' 필드가 1인 문서만 필터링합니다.
results = db.max_marginal_relevance_search("foo", filter=dict(page=1))

for doc in results:
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}")

[Content] foo, [metadata] {'page': 1}
[Content] bar, [metadata] {'page': 1}


`similarity_search` 함수를 호출할 때 `fetch_k` 매개변수를 설정하는 방법에 대한 예시입니다.

일반적으로 `fetch_k` 매개변수는 `k` 매개변수보다 훨씬 큰 값으로 설정하는 것이 좋습니다. 그 이유는 `fetch_k` 매개변수가 필터링 전에 가져올 문서의 수를 나타내기 때문입니다.

만약 `fetch_k`를 낮은 값으로 설정하면, 필터링할 문서가 충분하지 않을 수 있습니다.


In [106]:
results = db.similarity_search(
    "foo",  # 검색 쿼리
    # 메타데이터의 'page' 필드가 1인 문서만 필터링
    filter=dict(page=1),
    k=1,  # 가장 유사한 1개의 문서를 반환
    fetch_k=4,
)  # 4개의 문서까지 검색

for doc in results:
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}")

[Content] foo, [metadata] {'page': 1}


## 문서 삭제

벡터 저장소에 저장된 문서를 삭제할 수도 있습니다. 이때 삭제할 문서의 ID는 DB 에 저장된 ID와 일치해야 합니다.


`db.delete()` 메서드를 사용하여 문서를 삭제합니다.


In [118]:
db.index_to_docstore_id[0]

'ae2212d7-a646-469a-bbca-d54f5f34bd75'

In [116]:
# 인덱스 0에 해당하는 문서 저장소 ID를 사용하여 데이터베이스에서 문서를 삭제합니다.
db.delete([db.index_to_docstore_id[0]])

True